In [1]:
from nba_api.stats.static import players, teams
from nba_api.stats.endpoints import commonplayerinfo, playercareerstats, teamgamelog
import pandas as pd

In [3]:
nba_players = players.get_players()
df_players = pd.DataFrame(nba_players)
print(df_players.head())  # Show the first few players


      id            full_name first_name     last_name  is_active
0  76001       Alaa Abdelnaby       Alaa     Abdelnaby      False
1  76002      Zaid Abdul-Aziz       Zaid    Abdul-Aziz      False
2  76003  Kareem Abdul-Jabbar     Kareem  Abdul-Jabbar      False
3     51   Mahmoud Abdul-Rauf    Mahmoud    Abdul-Rauf      False
4   1505    Tariq Abdul-Wahad      Tariq   Abdul-Wahad      False


In [5]:
lebron = [player for player in nba_players if player["full_name"] == "LeBron James"]
print(lebron)


[{'id': 2544, 'full_name': 'LeBron James', 'first_name': 'LeBron', 'last_name': 'James', 'is_active': True}]


In [7]:
from nba_api.stats.endpoints import commonplayerinfo

player_id = 2544  # LeBron James
player_info = commonplayerinfo.CommonPlayerInfo(player_id=player_id)

df_info = player_info.get_data_frames()[0]
print(df_info)


   PERSON_ID FIRST_NAME LAST_NAME DISPLAY_FIRST_LAST DISPLAY_LAST_COMMA_FIRST  \
0       2544     LeBron     James       LeBron James            James, LeBron   

  DISPLAY_FI_LAST   PLAYER_SLUG            BIRTHDATE  \
0        L. James  lebron-james  1984-12-30T00:00:00   

                         SCHOOL COUNTRY  ...    PLAYERCODE FROM_YEAR TO_YEAR  \
0  St. Vincent-St. Mary HS (OH)     USA  ...  lebron_james      2003    2024   

   DLEAGUE_FLAG NBA_FLAG GAMES_PLAYED_FLAG DRAFT_YEAR DRAFT_ROUND  \
0             N        Y                 Y       2003           1   

   DRAFT_NUMBER GREATEST_75_FLAG  
0             1                Y  

[1 rows x 33 columns]


In [9]:
from nba_api.stats.endpoints import playercareerstats

career_stats = playercareerstats.PlayerCareerStats(player_id=2544)

df_career = career_stats.get_data_frames()[0]
print(df_career)


    PLAYER_ID SEASON_ID LEAGUE_ID     TEAM_ID TEAM_ABBREVIATION  PLAYER_AGE  \
0        2544   2003-04        00  1610612739               CLE        19.0   
1        2544   2004-05        00  1610612739               CLE        20.0   
2        2544   2005-06        00  1610612739               CLE        21.0   
3        2544   2006-07        00  1610612739               CLE        22.0   
4        2544   2007-08        00  1610612739               CLE        23.0   
5        2544   2008-09        00  1610612739               CLE        24.0   
6        2544   2009-10        00  1610612739               CLE        25.0   
7        2544   2010-11        00  1610612748               MIA        26.0   
8        2544   2011-12        00  1610612748               MIA        27.0   
9        2544   2012-13        00  1610612748               MIA        28.0   
10       2544   2013-14        00  1610612748               MIA        29.0   
11       2544   2014-15        00  1610612739       

In [11]:
import pandas as pd
from nba_api.stats.endpoints import playercareerstats

# LeBron James' Player ID
player_id = 2544  

# Fetch Career Stats
career_stats = playercareerstats.PlayerCareerStats(player_id=player_id)

# Convert to DataFrame
df_lebron = career_stats.get_data_frames()[0]

# Save to CSV
df_lebron.to_csv("lebron_stats.csv", index=False)

print("LeBron James' stats saved to lebron_stats.csv!")


LeBron James' stats saved to lebron_stats.csv!


In [13]:
from nba_api.stats.static import teams
from nba_api.stats.endpoints import commonteamroster
import pandas as pd

# Get Lakers' team ID
nba_teams = teams.get_teams()
lakers = [team for team in nba_teams if team['full_name'] == 'Los Angeles Lakers'][0]
lakers_team_id = lakers['id']

# Fetch Lakers' current roster
roster = commonteamroster.CommonTeamRoster(team_id=lakers_team_id)

# Convert to DataFrame
df_lakers_roster = roster.get_data_frames()[0]

# Save to CSV
df_lakers_roster.to_csv("lakers_roster.csv", index=False)

print("Lakers roster saved to lakers_roster.csv!")
print(df_lakers_roster)


Lakers roster saved to lakers_roster.csv!
        TeamID SEASON LeagueID               PLAYER   NICKNAME  \
0   1610612747   2024       00    Jarred Vanderbilt     Jarred   
1   1610612747   2024       00        Dalton Knecht     Dalton   
2   1610612747   2024       00          Cam Reddish        Cam   
3   1610612747   2024       00         Gabe Vincent       Gabe   
4   1610612747   2024       00         Bronny James     Bronny   
5   1610612747   2024       00     Christian Koloko  Christian   
6   1610612747   2024       00         Jaxson Hayes     Jaxson   
7   1610612747   2024       00          Maxi Kleber       Maxi   
8   1610612747   2024       00        Austin Reaves     Austin   
9   1610612747   2024       00  Dorian Finney-Smith     Dorian   
10  1610612747   2024       00         Shake Milton      Shake   
11  1610612747   2024       00         LeBron James     LeBron   
12  1610612747   2024       00             Alex Len       Alex   
13  1610612747   2024       00    

In [17]:
from nba_api.stats.endpoints import playergamelog, commonplayerinfo
import pandas as pd

# Load the Lakers roster dataset
file_path = "lakers_roster.csv"  # Ensure this file is in the same directory
df_lakers_roster = pd.read_csv(file_path)

# Extract player names and IDs from the roster dataset
lakers_players = df_lakers_roster[['PLAYER', 'PLAYER_ID']].dropna().to_dict('records')

# Fetch injury-related game logs for each Lakers player
all_injuries = []

for player in lakers_players:
    player_name = player["PLAYER"]
    player_id = int(player["PLAYER_ID"])

    # Fetch player info to determine their rookie season
    player_info = commonplayerinfo.CommonPlayerInfo(player_id=player_id)
    df_info = player_info.get_data_frames()[0]
    rookie_year = df_info.loc[0, "FROM_YEAR"]  # Extract rookie year

    # Fetch game logs from rookie year to current season (2023-24)
    for season in range(int(rookie_year), 2024):
        season_str = f"{season}-{str(season+1)[-2:]}"
        game_log = playergamelog.PlayerGameLog(player_id=player_id, season=season_str)
        df_games = game_log.get_data_frames()[0]

        if not df_games.empty:
            df_games["Player"] = player_name
            df_games["Season"] = season_str
            all_injuries.append(df_games)

# Combine all data into a DataFrame
df_lakers_injuries = pd.concat(all_injuries, ignore_index=True)

# Save to CSV
df_lakers_injuries.to_csv("lakers_injury_logs.csv", index=False)
print("Lakers injury game logs saved to lakers_injury_logs.csv!")


ReadTimeout: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)

In [19]:
import time
import pandas as pd
from nba_api.stats.endpoints import playercareerstats, playergamelog, commonplayerinfo
from nba_api.stats.static import players

# Load Lakers roster
file_path = "lakers_roster.csv"  # Ensure this file is in the same directory
df_lakers_roster = pd.read_csv(file_path)

# Extract Player Names and IDs
lakers_players = df_lakers_roster[['PLAYER', 'PLAYER_ID']].dropna().to_dict('records')

# Containers for data
all_career_stats = []
all_injury_logs = []

# Function to fetch career stats for each player
def fetch_career_stats(player_id, player_name):
    try:
        career_stats = playercareerstats.PlayerCareerStats(player_id=player_id)
        df_career = career_stats.get_data_frames()[0]
        df_career["Player"] = player_name
        return df_career
    except Exception as e:
        print(f"Error fetching career stats for {player_name}: {e}")
        return None

# Function to fetch injury-related logs (if possible)
def fetch_injury_logs(player_id, player_name):
    injury_logs = []
    try:
        player_info = commonplayerinfo.CommonPlayerInfo(player_id=player_id)
        df_info = player_info.get_data_frames()[0]
        rookie_year = int(df_info.loc[0, "FROM_YEAR"])  # Extract rookie year

        for season in range(rookie_year, 2024):
            season_str = f"{season}-{str(season+1)[-2:]}"
            game_log = playergamelog.PlayerGameLog(player_id=player_id, season=season_str)
            df_games = game_log.get_data_frames()[0]

            if not df_games.empty:
                df_games["Player"] = player_name
                df_games["Season"] = season_str
                injury_logs.append(df_games)

            time.sleep(1.5)  # Delay to prevent API rate limit

    except Exception as e:
        print(f"Error fetching injury logs for {player_name}: {e}")

    return pd.concat(injury_logs, ignore_index=True) if injury_logs else None

# Loop through each player and fetch data
for player in lakers_players:
    player_name = player["PLAYER"]
    player_id = int(player["PLAYER_ID"])
    
    print(f"Fetching stats for {player_name}...")

    # Fetch career stats
    career_stats = fetch_career_stats(player_id, player_name)
    if career_stats is not None:
        all_career_stats.append(career_stats)

    # Fetch injury logs
    injury_logs = fetch_injury_logs(player_id, player_name)
    if injury_logs is not None:
        all_injury_logs.append(injury_logs)

    # Add a delay to prevent API blocks
    time.sleep(2)

# Combine and save data
if all_career_stats:
    df_all_career_stats = pd.concat(all_career_stats, ignore_index=True)
    df_all_career_stats.to_csv("lakers_career_stats.csv", index=False)
    print("Lakers Career Stats saved as 'lakers_career_stats.csv'")

if all_injury_logs:
    df_all_injury_logs = pd.concat(all_injury_logs, ignore_index=True)
    df_all_injury_logs.to_csv("lakers_injury_logs.csv", index=False)
    print("Lakers Injury Logs saved as 'lakers_injury_logs.csv'")


Fetching stats for Jarred Vanderbilt...
Error fetching career stats for Jarred Vanderbilt: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)
Fetching stats for Dalton Knecht...
Fetching stats for Cam Reddish...
Fetching stats for Gabe Vincent...
Fetching stats for Bronny James...
Fetching stats for Christian Koloko...
Fetching stats for Jaxson Hayes...
Fetching stats for Maxi Kleber...
Fetching stats for Austin Reaves...
Fetching stats for Dorian Finney-Smith...
Fetching stats for Shake Milton...
Fetching stats for LeBron James...
Fetching stats for Alex Len...
Fetching stats for Rui Hachimura...
Fetching stats for Jordan Goodwin...
Fetching stats for Trey Jemison III...
Fetching stats for Luka Dončić...
Fetching stats for Markieff Morris...
Lakers Career Stats saved as 'lakers_career_stats.csv'
Lakers Injury Logs saved as 'lakers_injury_logs.csv'


In [21]:
import time
import pandas as pd
from nba_api.stats.endpoints import playercareerstats, playergamelog, commonplayerinfo
from nba_api.stats.static import players

# Load Lakers roster
file_path = "lakers_roster.csv"  # Ensure this file is in the same directory
df_lakers_roster = pd.read_csv(file_path)

# Extract Player Names and IDs
lakers_players = df_lakers_roster[['PLAYER', 'PLAYER_ID']].dropna().to_dict('records')

# Containers for data
all_career_stats = []
all_injury_logs = []

# Function to fetch career stats for each player
def fetch_career_stats(player_id, player_name):
    try:
        career_stats = playercareerstats.PlayerCareerStats(player_id=player_id)
        df_career = career_stats.get_data_frames()[0]
        df_career["Player"] = player_name
        return df_career
    except Exception as e:
        print(f"Error fetching career stats for {player_name}: {e}")
        return None

# Function to fetch injury-related logs (if possible)
def fetch_injury_logs(player_id, player_name):
    injury_logs = []
    try:
        player_info = commonplayerinfo.CommonPlayerInfo(player_id=player_id)
        df_info = player_info.get_data_frames()[0]
        rookie_year = int(df_info.loc[0, "FROM_YEAR"])  # Extract rookie year

        for season in range(rookie_year, 2024):
            season_str = f"{season}-{str(season+1)[-2:]}"
            game_log = playergamelog.PlayerGameLog(player_id=player_id, season=season_str)
            df_games = game_log.get_data_frames()[0]

            if not df_games.empty:
                df_games["Player"] = player_name
                df_games["Season"] = season_str
                injury_logs.append(df_games)

            time.sleep(1.5)  # Delay to prevent API rate limit

    except Exception as e:
        print(f"Error fetching injury logs for {player_name}: {e}")

    return pd.concat(injury_logs, ignore_index=True) if injury_logs else None

# Loop through each player and fetch data
for player in lakers_players:
    player_name = player["PLAYER"]
    player_id = int(player["PLAYER_ID"])
    
    print(f"Fetching stats for {player_name}...")

    # Fetch career stats
    career_stats = fetch_career_stats(player_id, player_name)
    if career_stats is not None:
        all_career_stats.append(career_stats)

    # Fetch injury logs
    injury_logs = fetch_injury_logs(player_id, player_name)
    if injury_logs is not None:
        all_injury_logs.append(injury_logs)

    # Add a delay to prevent API blocks
    time.sleep(2)

# Combine and save data
if all_career_stats:
    df_all_career_stats = pd.concat(all_career_stats, ignore_index=True)
    df_all_career_stats.to_csv("lakers_career_stats.csv", index=False)
    print("Lakers Career Stats saved as 'lakers_career_stats.csv'")

if all_injury_logs:
    df_all_injury_logs = pd.concat(all_injury_logs, ignore_index=True)
    df_all_injury_logs.to_csv("lakers_injury_logs.csv", index=False)
    print("Lakers Injury Logs saved as 'lakers_injury_logs.csv'")


Fetching stats for Jarred Vanderbilt...
Error fetching career stats for Jarred Vanderbilt: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Fetching stats for Dalton Knecht...
Fetching stats for Cam Reddish...
Fetching stats for Gabe Vincent...
Fetching stats for Bronny James...
Fetching stats for Christian Koloko...
Fetching stats for Jaxson Hayes...
Fetching stats for Maxi Kleber...
Fetching stats for Austin Reaves...
Fetching stats for Dorian Finney-Smith...
Fetching stats for Shake Milton...
Fetching stats for LeBron James...
Fetching stats for Alex Len...
Fetching stats for Rui Hachimura...
Fetching stats for Jordan Goodwin...
Fetching stats for Trey Jemison III...
Fetching stats for Luka Dončić...
Fetching stats for Markieff Morris...
Lakers Career Stats saved as 'lakers_career_stats.csv'
Lakers Injury Logs saved as 'lakers_injury_logs.csv'


In [23]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from nba_api.stats.endpoints import playergamelog
import time

# Load Lakers roster (Ensure your lakers_roster.csv contains PLAYER & PLAYER_ID)
df_lakers_roster = pd.read_csv("lakers_roster.csv")

# Function to scrape ESPN injury data
def scrape_espn_injuries():
    url = "https://www.espn.com/nba/injuries"
    headers = {"User-Agent": "Mozilla/5.0"}
    
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")
    
    injury_data = []
    
    # Find injury table rows
    for row in soup.find_all("tr", class_="Table__TR"):
        cols = row.find_all("td")
        if len(cols) > 2:
            player = cols[0].text.strip()
            team = cols[1].text.strip()
            injury_desc = cols[2].text.strip()
            
            injury_data.append({"Player": player, "Team": team, "Injury_Description": injury_desc})

    return pd.DataFrame(injury_data)

# Function to fetch NBA API injury logs
def fetch_nba_injury_logs(player_id, player_name):
    injury_logs = []
    try:
        for season in range(2013, 2024):  # Adjust for full career history
            season_str = f"{season}-{str(season+1)[-2:]}"
            game_log = playergamelog.PlayerGameLog(player_id=player_id, season=season_str)
            df_games = game_log.get_data_frames()[0]
            
            # Filter games where the player did not play
            df_games["DNP"] = df_games["MIN"] == 0
            df_games["Player"] = player_name
            df_games["Season"] = season_str
            
            injury_logs.append(df_games[df_games["DNP"]])

            time.sleep(1.5)  # Prevent API rate limits
    except Exception as e:
        print(f"Error fetching game logs for {player_name}: {e}")
    
    return pd.concat(injury_logs, ignore_index=True) if injury_logs else None

# Fetch injury logs for Lakers players
all_injury_logs = []
for _, row in df_lakers_roster.iterrows():
    player_name = row["PLAYER"]
    player_id = int(row["PLAYER_ID"])
    print(f"Fetching injury logs for {player_name}...")
    
    injury_logs = fetch_nba_injury_logs(player_id, player_name)
    if injury_logs is not None:
        all_injury_logs.append(injury_logs)

# Combine all injury logs
if all_injury_logs:
    df_injury_logs = pd.concat(all_injury_logs, ignore_index=True)
    df_injury_logs.to_csv("nba_injury_logs.csv", index=False)
    print("NBA Injury Logs saved as 'nba_injury_logs.csv'")

# Scrape ESPN for injury descriptions
df_espn_injuries = scrape_espn_injuries()
df_espn_injuries.to_csv("espn_injury_reports.csv", index=False)
print("ESPN Injury Reports saved as 'espn_injury_reports.csv'")

# Merge NBA API logs with scraped ESPN injury data
df_merged = df_injury_logs.merge(df_espn_injuries, on="Player", how="left")
df_merged.to_csv("merged_injury_data.csv", index=False)
print("Merged injury data saved as 'merged_injury_data.csv'")


Fetching injury logs for Jarred Vanderbilt...


/var/folders/62/2x3kjfjn66l6j6fwjpnc55m00000gn/T/ipykernel_10739/2961022715.py:52: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat(injury_logs, ignore_index=True) if injury_logs else None


Fetching injury logs for Dalton Knecht...
Fetching injury logs for Cam Reddish...
Fetching injury logs for Gabe Vincent...


/var/folders/62/2x3kjfjn66l6j6fwjpnc55m00000gn/T/ipykernel_10739/2961022715.py:52: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat(injury_logs, ignore_index=True) if injury_logs else None


Fetching injury logs for Bronny James...
Fetching injury logs for Christian Koloko...
Fetching injury logs for Jaxson Hayes...
Fetching injury logs for Maxi Kleber...


...

Fetching injury logs for Shake Milton...
Fetching injury logs for LeBron James...
Fetching injury logs for Alex Len...
Fetching injury logs for Rui Hachimura...
Fetching injury logs for Jordan Goodwin...


/var/folders/62/2x3kjfjn66l6j6fwjpnc55m00000gn/T/ipykernel_10739/2961022715.py:52: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat(injury_logs, ignore_index=True) if injury_logs else None


Fetching injury logs for Trey Jemison III...
Fetching injury logs for Luka Dončić...
Fetching injury logs for Markieff Morris...


/var/folders/62/2x3kjfjn66l6j6fwjpnc55m00000gn/T/ipykernel_10739/2961022715.py:67: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_injury_logs = pd.concat(all_injury_logs, ignore_index=True)


NBA Injury Logs saved as 'nba_injury_logs.csv'
ESPN Injury Reports saved as 'espn_injury_reports.csv'
Merged injury data saved as 'merged_injury_data.csv'
